# Projeto - Case VAI Shoes
##0 - Instalando e importando bibliotecas
Nesta etapa instalamos e importamos as principais bibliotecas utilizadas na construção do notebook.

In [ ]:
# Import das libs utilizadas

# Manipulação de dados
import pickle
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
from textwrap import wrap
# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
# Modelagem - Clusterização
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
# Modelagem - Regressão
# Modelagem - Série Temporal
from keras.models import Sequential #!pip install keras e !pip install tensorflow
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import StandardScaler #!pip install sklearn
from keras.callbacks import ModelCheckpoint
#Importando fbprophet
from prophet import Prophet #maneira de instalação e importação mudou recentemente
# Modelagem - Análise de cesta

In [ ]:
# Definindo função que fornece a quantidade e os valores único de cada feature de um dataframe
def get_values_unique(df):
    """"
    Função que, fornecido um DataFrame, printa a quantidade de valores
    únicos e os valores únicos de cada feature.

    In: DataFrame.

    Out: Print da quantidade e valores únicos da feature.
    """
    for column in df.columns:
        print(f'{column} possui {df[column].nunique()} valores únicos:')
        print(f'\n{df[column].unique()}')
        print('\n*********************************************\n')

## 1 - Aquisição dos dados

### a) Carregamento das bases de dados

In [ ]:
# Lê arquivos ----------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')


MessageError: ignored

### b) Visualização dos DataFrames e suas informações

### c) Visualização dos valores únicos dos DataFrames

In [ ]:
# Visualizando valores únicos das features de df_vendas
get_values_unique(df_vendas)

In [ ]:
# Visualizando valores únicos das features de df_previs
get_values_unique(df_previs)

### Pontos sobre a análise inicial dos dados.


Tendências no mercado calçadista:

Sustentabilidade: Há uma crescente demanda por calçados produzidos de forma sustentável, com materiais recicláveis ou de origem responsável. Os consumidores estão mais conscientes sobre as questões ambientais e buscam opções ecologicamente corretas.
Conforto e funcionalidade: Os consumidores têm dado preferência a calçados que oferecem conforto, suporte e durabilidade. Calçados com tecnologias inovadoras, como solados ergonômicos e materiais respiráveis, têm ganhado destaque.
Customização: A personalização de calçados tem se tornado uma tendência relevante, permitindo que os consumidores criem produtos únicos de acordo com suas preferências e estilo.

Fatores de impacto:

Economia: O desempenho econômico do país tem um impacto significativo no mercado calçadista. Em momentos de recessão, por exemplo, as vendas podem diminuir devido à redução do poder de compra dos consumidores.
Moda e estilo: Tendências de moda e estilo influenciam a demanda por calçados. A indústria deve estar atenta às preferências e mudanças de comportamento dos consumidores para se adaptar e oferecer produtos atualizados.
Custo dos materiais: O preço e a disponibilidade de matérias-primas, como couro, borracha e tecidos, afetam diretamente os custos de produção e, consequentemente, os preços dos calçados.

Concorrência internacional: A competição com produtos importados, especialmente da Ásia, é um desafio para a indústria calçadista brasileira. As diferenças de custo de produção e acordos comerciais podem impactar a competitividade dos calçados nacionais.

_"O ano de 2020 foi marcado pela pandemia, resultando em uma queda de 18,4% na produção
brasileira de calçados. A indústria calçadista foi impactada por meio de diversos vetores. Um
levantamento realizado pela Abicalçados em fevereiro de 2021 sobre a percepção das empresas
quanto aos impactos da Covid-19 nos seus negócios destacou que, em 2020, o fechamento do
comércio, a queda do faturamento e a redução da demanda foram os principais vetores de impactos da pandemia sobre a indústria calçadista."_

_Informações Adicionais:_

* _Brasil está no TOP 5 produtores mundiais_
* _Mulheres dominam o mercado calçadista (70% dele)_

## 2 - Análise Exploratório dos Dados

In [ ]:
#Qual é o total de vendas por mês?
vendas_mensais = pd.read_csv(r'/content/drive/MyDrive/VAI_SHOES/dados/tbl_vendas_mensais.csv', sep=';')

_A base de dados vai de Jan/2019 a Mar/2023_

In [ ]:
vendas_mensais['vl_sale_price'] = vendas_mensais['vl_sale_price'].apply(lambda x: float(str(x).replace(',', '.')))

_Houve um possivel erro ao carregar os dados de Novembro de 2021, os dados estão duplicados mas com valores de vendas diferentes, para contornar esse problema, vamos utilizar a média entre os dois valores_

In [ ]:
vendas_mensais = vendas_mensais.groupby(['dt_sale','id_sku','ds_sku','ds_category','ds_product_line','ds_brand_segment','ds_tecnology']).mean().reset_index()
sop = sop.groupby(['dt_sale','ds_product_line']).mean().reset_index()

_Algumas datas possuem formatação diferente, então para realizar seu tratamento, primeiro vamos transforma-las em data e depois mudar o formato com que elas são exibidas. Como os dados são mensais, vamos deixar a formatação com o primeiro dia do mês, o mês correspondente e o ano_

In [ ]:
vendas_mensais.head(2)

_Procurando por valores nulos, encontramos que a coluna "vl_sale_price" possui 24 valores nulos, para tentar justificar a presença desses valores vamos investigar para avaliar o tipo de tratamento que será utilizado

Coluna | Valores nulos
:---: | :---:
vl_sale_price | 24

_Encontramos que os dados vazios correspondem a 2 produtos: NYC Wave e NYC Sub. Que pertencem a mesma categoria, linha, marca e usam a mesma tecnologia. Possivelmente são 2 produtos que foram adicionados recentemente a base de dados da empresa (Abril de 2022)_

_Encontramos que os dados vazios correspondem a 2 produtos: NYC Wave e NYC Sub. Que pertencem a mesma categoria, linha, marca e usam a mesma tecnologia. Possivelmente são 2 produtos que foram adicionados recentemente a base de dados da empresa (Abril de 2022)_

In [ ]:
vendas_mensais[vendas_mensais.vl_sale_price.isnull()][['id_sku', 'ds_sku', 'ds_category', 'ds_product_line',
       'ds_brand_segment', 'ds_tecnology']].drop_duplicates()

In [ ]:
gera_vl_vazios = vendas_mensais.\
  query('ds_category=="NYCs Shoes" and ds_product_line=="Shoes" and ds_brand_segment=="Core" and ds_tecnology=="LiteWeave"').\
  dropna()

gera_vl_vazios['unit_price'] = gera_vl_vazios['vl_sale_price']/gera_vl_vazios['qt_sale']

In [ ]:
px.line(gera_vl_vazios, x='dt_sale', y='unit_price')

_Como o preço desse conjunto permanece constante no periodo em que esses 2 produtos foram inclusos na base, vamos selecionar os dados que correspondem a média do preço dos produtos da mesma classe e utiliza-los para preencher os valores vazios_

In [ ]:
preco_novos_calcados = gera_vl_vazios.query("dt_sale >= '2022-04-01'").unit_price.mean()

In [ ]:
vendas_mensais['vl_sale_price'] = vendas_mensais.apply(lambda x: x['qt_sale'] * preco_novos_calcados if pd.isna(x['vl_sale_price']) else x['vl_sale_price'], axis=1)

_Outro problema observado na base de dados é a questão das classes **SNEARKER**  e **SNEAKERS**, que representam o mesmo grupo. Então para corrigir esse problema vamos uni-los_

In [ ]:
fig = px.bar(vendas_mensais.groupby(vendas_mensais['dt_sale'].dt.year).agg({'qt_sale':'sum'}).reset_index(),\
              x='dt_sale', y='qt_sale')

fig.update_layout(xaxis_title="Período", yaxis_title="QTD. Vendida", title='Vendas Anuais')

fig.show()

In [ ]:
fig = px.bar(vendas_mensais.groupby(vendas_mensais['dt_sale'].dt.year).agg({'vl_sale_price':'sum'}).reset_index(),\
              x='dt_sale', y='vl_sale_price')

fig.update_layout(xaxis_title="Período",  title='Faturamento Anual', yaxis_title='Vendas')

fig.show()

In [ ]:
#Analisando a série temporal como um todo
fig = px.line(vendas_mensais.groupby('dt_sale').agg({'qt_sale':'sum'}).reset_index(),\
              x='dt_sale', y='qt_sale')

fig.update_layout(xaxis_title="Período", yaxis_title="QTD. Vendida")

fig.show()

In [ ]:
#Agora analisando as series de cada uma das classes de produtos
fig = px.line(vendas_mensais.groupby([vendas_mensais.dt_sale, 'ds_product_line']).agg({'qt_sale':'sum'}).reset_index(),\
              x='dt_sale', y='qt_sale', color='ds_product_line')

fig.update_layout(xaxis_title="Período", yaxis_title="QTD. Vendida")

fig.show()

In [ ]:
fig = px.line(vendas_mensais.groupby([vendas_mensais.dt_sale, 'ds_sku']).agg({'unit_price':'mean'}).reset_index(),\
              x='dt_sale', y='unit_price', color='ds_sku')

fig.update_layout(xaxis_title="Período", yaxis_title="Preço", title='Preço médio por produto')

fig.show()

In [ ]:
fig = px.bar(vendas_mensais.groupby(['ds_sku']).agg({'qt_sale':'sum'}).reset_index().sort_values('qt_sale', ascending=False),\
              x='ds_sku', y='qt_sale', color='ds_sku')

fig.update_layout(xaxis_title="Período", yaxis_title="Quantidade", title='Quantidade de produtos vendidos')

fig.show()

In [ ]:
#Observa-se que os produtos mais vendidos são: Machina Metal, Max e Nautical Classic
fig = px.bar(vendas_mensais.groupby(['ds_sku']).agg({'vl_sale_price':'sum'}).reset_index().sort_values('vl_sale_price', ascending=False),\
              x='ds_sku', y='vl_sale_price', color='ds_sku')

fig.update_layout(xaxis_title="Período", yaxis_title="Vendas", title='Vendas Geradas')

fig.show()

_Quando olhamos as vendas o produto que gera maior capital é o Machina (TOP 1) seguido do Nautical Classic (TOP 3) por ultimo o Classic (TOP 5)_

_O preço unitário médio de cada classe apresenta flutu_

_Agora vamos realizar uma análise estatística dos dados_

In [ ]:
data = vendas_mensais.groupby(['dt_sale']).agg({'qt_sale':'sum'}).reset_index().qt_sale

# Calcule os valores de ACF e PACF
lags = 12
acf_values = acf(data, nlags=lags)
pacf_values = pacf(data, nlags=lags)

# Crie os gráficos ACF e PACF como scatter plots com linhas individuais
fig_acf = go.Figure(data=[
    go.Scatter(x=np.arange(len(acf_values)), y=acf_values, mode='markers', name='ACF'),
])

for i in range(len(acf_values)):
    fig_acf.add_shape(type='line',
                      x0=i, y0=acf_values[i],
                      x1=i, y1=0,
                      line=dict(color='black', width=1))
fig_acf.add_shape(
    type='line',
    x0=0, y0=-0.5,
    x1=len(acf_values), y1=-0.5,
    line=dict(color='red', width=1, dash='dash')
)

fig_acf.add_shape(
    type='line',
    x0=0, y0=0.5,
    x1=len(acf_values), y1=0.5,
    line=dict(color='red', width=1, dash='dash'))

fig_acf.update_layout(
    title="Autocorrelation Function (ACF)",
    xaxis_title="Lag",
    yaxis_title="ACF Value"
)

fig_pacf = go.Figure(data=[
    go.Scatter(x=np.arange(len(pacf_values)), y=pacf_values, mode='markers', name='PACF'),
])

for i in range(len(pacf_values)):
    fig_pacf.add_shape(type='line',
                       x0=i, y0=pacf_values[i],
                       x1=i, y1=0,
                       line=dict(color='black', width=1))
fig_pacf.add_shape(
    type='line',
    x0=0, y0=-0.25,
    x1=len(acf_values), y1=-0.25,
    line=dict(color='red', width=1, dash='dash')
)

fig_pacf.add_shape(
    type='line',
    x0=0, y0=0.25,
    x1=len(acf_values), y1=0.25,
    line=dict(color='red', width=1, dash='dash'))

fig_pacf.update_layout(
    title="Partial Autocorrelation Function (PACF)",
    xaxis_title="Lag",
    yaxis_title="PACF Value"
)

# Exiba os gráficos
fig_acf.show()
fig_pacf.show()


_Pelo grafico ACF podemos observar que nosso problema se trata de uma serie temporal não estacionária que possui autocorrelação. Além disso, ele reforça a ideia da sazonalidade_

_Como os spikes do gráfico PACF (Partial Autocorrelate Function) ultrapassam o intervalo demarcado, isso indica que a série possue autocorrelação e é não estacionária. Outro indicativo de que a série é autocorrelacionada é a presença de uma queda suave no gráfico ACF(Autocorrelate Function)_

In [ ]:
data = vendas_mensais.groupby(['dt_sale']).agg({'qt_sale':'sum'}).reset_index()
data = data.set_index('dt_sale')
decomposition = seasonal_decompose(data['qt_sale'], model='additive', period=12)

fig = make_subplots(rows=4, cols=1, shared_xaxes=True)

# Criação dos gráficos separados para cada componente
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)

# Série temporal
fig.add_trace(go.Scatter(x=data.index, y=data['qt_sale'], name='Série Temporal'), row=1, col=1)
fig.update_yaxes(title_text='Quantidade Vendida', row=1, col=1)

# Sazonalidade
fig.add_trace(go.Scatter(x=data.index, y=decomposition.seasonal, name='Sazonalidade'), row=2, col=1)
fig.update_yaxes(title_text='Sazonalidade', row=2, col=1)

# Tendência
fig.add_trace(go.Scatter(x=data.index, y=decomposition.trend, name='Tendência'), row=3, col=1)
fig.update_yaxes(title_text='Tendência', row=3, col=1)

# Resíduos
fig.add_trace(go.Scatter(x=data.index, y=decomposition.resid,mode='markers', marker=dict(color='purple'), name='Resíduos'), row=4, col=1)
fig.update_yaxes(title_text='Resíduos', row=4, col=1)

fig.update_layout(height=800, title_text='Decomposição da Série Temporal')

fig.show()

_Nosso problema se baseia em uma série temporal com tendência positiva e observa-se a presença de uma sazonalidade. Em relação aos residuos, os mesmo apresentam média zero, distribuição proxima a normalidade, ainda precisa avaliar a questão da homocedasticidade_

In [ ]:
# Gerar a função KDE
kde_x = np.linspace(decomposition.resid.min(), decomposition.resid.max(), 100)
media = np.nanmean(decomposition.resid)
residuos_preenchidos = [valor if np.isfinite(valor) else media for valor in decomposition.resid]
kde_y = gaussian_kde(residuos_preenchidos)(kde_x)*30

# Criar o histograma com a função KDE
fig = go.Figure()
fig.add_trace(go.Histogram(x=decomposition.resid, histnorm='density', name='Histograma'))
fig.add_trace(go.Scatter(x=kde_x, y=kde_y, mode='lines', name='KDE'))

# Atualizar o layout do gráfico
fig.update_layout(
    title="Histograma com KDE",
    xaxis_title="Valores",
    yaxis_title="Densidade"
)

# Exibir o gráfico
fig.show()

## 3 - Limpeza e pré-processamento de dados

In [ ]:
# Importando os dados
df = pd.read_csv('/content/tbl_vendas_mensais.csv', sep=';')

# Converte a coluna 'dt_sale' para tipo datetime.
# Joga fora as colunas não relevantes
# Converte a categoria de Sneaker para Sneakers
df['dt_sale'] = pd.to_datetime(df['dt_sale'], utc=True)
df.drop(columns=['ds_sku', 'ds_category', 'ds_brand_segment', 'ds_tecnology', 'vl_sale_price'], inplace=True)
df.loc[df['ds_product_line'] == 'Sneaker', 'ds_product_line'] = 'Sneakers'

# Faz a média nos valores de novembro de 2021
df_novembro_2021 = df[df['dt_sale'] == '2021-11']
df_novembro_2021 = df_novembro_2021.groupby(['id_sku', 'ds_product_line', 'dt_sale']).mean()
df_novembro_2021.reset_index(inplace=True)
df_novembro_2021 = df_novembro_2021.set_index('dt_sale')

# Remove os valores antigos de novembro de 2021 e adiciona os novos
df.drop(index=df[df['dt_sale'] == '2021-11'].index, inplace=True)
df.set_index('dt_sale', drop=True, inplace=True)
df = pd.concat([df, df_novembro_2021])
df.drop(columns='id_sku', inplace=True)
df.sort_index(inplace=True)

# Retorna o dataset final. Para cada produto('ds_product_line') temos a quantidade total de vendas.
df = df.groupby(['dt_sale', 'ds_product_line']).sum()
df.reset_index(inplace=True)
df

,dt_sale,ds_product_line,qt_sale
0,2019-01-01 00:00:00+00:00,Boot,191839.0
1,2019-01-01 00:00:00+00:00,Sandals,311140.0
2,2019-01-01 00:00:00+00:00,Shoes,575501.0
3,2019-01-01 00:00:00+00:00,Sneakers,279316.0
4,2019-02-01 00:00:00+00:00,Boot,214879.0
...,...,...,...
199,2023-02-01 00:00:00+00:00,Sneakers,511301.0
200,2023-03-01 00:00:00+00:00,Boot,184398.0
201,2023-03-01 00:00:00+00:00,Sandals,481353.0
202,2023-03-01 00:00:00+00:00,Shoes,812154.0


# **4 - Implementação dos Modelos**

## **Rede Neural Recorrente (arquitetura LSTM)**


In [ ]:
df = df.loc[:, ['dt_sale', 'ds_product_line', 'qt_sale']]

tbl = df.groupby(['dt_sale', 'ds_product_line']).sum()
tbl = tbl.sort_values(by='dt_sale')
tbl = tbl.reset_index()

#Separando o Df de cada categoria
df_Boot = tbl[tbl['ds_product_line'] == 'Boot']
df_Sandals = tbl[tbl['ds_product_line'] == 'Sandals']
df_Shoes = tbl[tbl['ds_product_line'] == 'Shoes']
df_Sneakers = tbl[tbl['ds_product_line'] == 'Sneakers']

In [ ]:
#Função que separa os atributos previsores para treinamento da rede neural
def Sepacao_previsores(df):
    df_treino = df.iloc[:47,:]

    #Escalonando
    df_treino = df_treino['qt_sale'].values
    df_treino = df_treino.reshape(-1, 1)
    escaler = StandardScaler()
    df_treino_escalonada = escaler.fit_transform(df_treino)

    previsores = []
    qt_real = []

    #Para a previsão de um dia, nesse caso, são necessários analisar os 12 meses anteriores
    for i in range(12, len(df_treino)):
        previsores.append(df_treino_escalonada[i - 12:i, 0])
        qt_real.append(df_treino_escalonada[i, 0])

    #Reformatação para a forma requerida pelo KERAS
    previsores, qt_real = np.array(previsores), np.array(qt_real)
    previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))
    return previsores, qt_real, escaler

In [ ]:
#Função que aplica a rede neural
def RNN(previsores, qt_real):
    #Arquitetura da rede neural
    regressor = Sequential()
    regressor.add(LSTM(units=100, return_sequences=True, input_shape=(previsores.shape[1], 1)))
    regressor.add(Dropout(0.3)) #Irá zerar 30% das entradas (Ajuda a previnir overfitting)

    regressor.add(LSTM(units=40, return_sequences=True))
    regressor.add(Dropout(0.3))

    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.3))

    regressor.add(LSTM(units=60, return_sequences=False))
    regressor.add(Dropout(0.2))

    #Camada de saída
    regressor.add(Dense(units= 1, activation='linear'))

    regressor.compile(optimizer='rmsprop', loss='mean_squared_error', metrics = ['mean_absolute_error'])

    #mc = ModelCheckpoint('Best_weights.h5', save_best_only=True, monitor='loss', verbose=1) #Salva os melhores pesos da RN

    regressor.fit(previsores, qt_real, epochs=200, batch_size=12)
    return regressor

In [ ]:
#função que a partir do modelo treinado cria por meio de condicionais as colunas m1, m2 e m3
def Previsões(df, regressor, escaler):
    m1 = []
    m2 = []
    m3 = []
    #Ciclo que vai de setembro até fevereiro
    for i in range(45, 51):
        #Previsão de Setembro
        if i == 45:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 2:
                    prev = escaler.inverse_transform(prev)
                    m3.append(float(prev))
                entradas = np.delete(entradas, 0, axis=0) #Retro alimentação, excluindo o primeiro...
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0) #E adicionando a previsão com ultimo
        #Previsão de Outubro
        if i == 46:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 1: #M - 2
                   prev_inverse = escaler.inverse_transform(prev)
                   m2.append(float(prev_inverse))
                if a == 2: #M - 3
                    prev = escaler.inverse_transform(prev)
                    m3.append(float(prev))
                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)
        #Previsão de Novembro
        if i == 47:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 0: #M - 1
                   prev_inverse = escaler.inverse_transform(prev)
                   m1.append(float(prev_inverse))
                if a == 1: #M - 2
                   prev_inverse = escaler.inverse_transform(prev)
                   m2.append(float(prev_inverse))
                if a == 2: #M - 3
                    prev = escaler.inverse_transform(prev)
                    m3.append(float(prev))
                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)
        #Previsão de Dezembro
        if i == 48:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 0: #M - 1
                   prev_inverse = escaler.inverse_transform(prev)
                   m1.append(float(prev_inverse))
                if a == 1: #M - 2
                   prev_inverse = escaler.inverse_transform(prev)
                   m2.append(float(prev_inverse))
                if a == 2: #M - 3
                    prev = escaler.inverse_transform(prev)
                    m3.append(float(prev))
                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)
        #Previsão de Janeiro
        if i == 49:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 0: #M - 1
                   prev_inverse = escaler.inverse_transform(prev)
                   m1.append(float(prev_inverse))
                if a == 1: #M - 2
                   prev_inverse = escaler.inverse_transform(prev)
                   m2.append(float(prev_inverse))

                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)
        #Previsão de Fevereiro
        if i == 50:
            df_teste = df.iloc[i,:]
            df_B = df['qt_sale']
            entradas = df_B[i - 12:i].values
            entradas = entradas.reshape(-1, 1)
            entradas = escaler.transform(entradas)

            for a in range(0, 3):
                X_teste = []
                X_teste.append(entradas[0: 12, 0])
                X_teste = np.array(X_teste)
                X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
                prev = regressor.predict(X_teste)

                if a == 0: #M - 1
                   prev_inverse = escaler.inverse_transform(prev)
                   m1.append(float(prev_inverse))

                entradas = np.delete(entradas, 0, axis=0)
                entradas = np.insert(entradas, entradas.shape[0], [prev],axis= 0)

    Product = df.iloc[47:]

    Product['Previsão_M1'] = m1
    Product['Previsão_M2'] = m2
    Product['Previsão_M3'] = m3
    return Product

In [ ]:
#Chamado das funções e geração das tabelas de previsão
previsores, qt_real, escaler = Sepacao_previsores(df_Boot)
Boot = Previsões(df_Boot, RNN(previsores, qt_real), escaler)

previsores, qt_real, escaler = Sepacao_previsores(df_Sandals)
Sandals = Previsões(df_Sandals, RNN(previsores, qt_real), escaler)

previsores, qt_real, escaler = Sepacao_previsores(df_Shoes)
Shoes = Previsões(df_Shoes, RNN(previsores, qt_real), escaler)

previsores, qt_real, escaler = Sepacao_previsores(df_Sneakers)
Sneakers = Previsões(df_Sneakers, RNN(previsores, qt_real), escaler)

Epoch 1/200
3/3 [==============================] - 11s 51ms/step - loss: 1.0051 - mean_absolute_error: 0.8580
Epoch 2/200
3/3 [==============================] - 0s 62ms/step - loss: 0.9873 - mean_absolute_error: 0.8591
Epoch 3/200
3/3 [==============================] - 0s 56ms/step - loss: 0.9429 - mean_absolute_error: 0.8416
Epoch 4/200
3/3 [==============================] - 0s 50ms/step - loss: 0.9700 - mean_absolute_error: 0.8617
Epoch 5/200
3/3 [==============================] - 0s 50ms/step - loss: 0.9363 - mean_absolute_error: 0.8422
Epoch 6/200
3/3 [==============================] - 0s 52ms/step - loss: 0.9657 - mean_absolute_error: 0.8657
Epoch 7/200
3/3 [==============================] - 0s 52ms/step - loss: 0.9180 - mean_absolute_error: 0.8375
Epoch 8/200
3/3 [==============================] - 0s 49ms/step - loss: 0.9059 - mean_absolute_error: 0.8333
Epoch 9/200
3/3 [==============================] - 0s 42ms/step - loss: 0.8883 - mean_absolute_error: 0.8270
Epoch 10/200
3/3 [

<ipython-input-35-2d5bc51efce8>:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Product['Previsão_M1'] = m1
<ipython-input-35-2d5bc51efce8>:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Product['Previsão_M2'] = m2
<ipython-input-35-2d5bc51efce8>:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

Epoch 1/200
3/3 [==============================] - 9s 36ms/step - loss: 0.5947 - mean_absolute_error: 0.6451
Epoch 2/200
3/3 [==============================] - 0s 39ms/step - loss: 0.3953 - mean_absolute_error: 0.5357
Epoch 3/200
3/3 [==============================] - 0s 36ms/step - loss: 0.3175 - mean_absolute_error: 0.4835
Epoch 4/200
3/3 [==============================] - 0s 38ms/step - loss: 0.3263 - mean_absolute_error: 0.4812
Epoch 5/200
3/3 [==============================] - 0s 40ms/step - loss: 0.3301 - mean_absolute_error: 0.4779
Epoch 6/200
3/3 [==============================] - 0s 47ms/step - loss: 0.3613 - mean_absolute_error: 0.5010
Epoch 7/200
3/3 [==============================] - 0s 40ms/step - loss: 0.3073 - mean_absolute_error: 0.4707
Epoch 8/200
3/3 [==============================] - 0s 41ms/step - loss: 0.3105 - mean_absolute_error: 0.4722
Epoch 9/200
3/3 [==============================] - 0s 39ms/step - loss: 0.3355 - mean_absolute_error: 0.4968
Epoch 10/200
3/3 [=

<ipython-input-35-2d5bc51efce8>:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Product['Previsão_M1'] = m1
<ipython-input-35-2d5bc51efce8>:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Product['Previsão_M2'] = m2
<ipython-input-35-2d5bc51efce8>:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

Epoch 1/200
3/3 [==============================] - 9s 32ms/step - loss: 0.8353 - mean_absolute_error: 0.7878
Epoch 2/200
3/3 [==============================] - 0s 32ms/step - loss: 0.7193 - mean_absolute_error: 0.7481
Epoch 3/200
3/3 [==============================] - 0s 34ms/step - loss: 0.6864 - mean_absolute_error: 0.7469
Epoch 4/200
3/3 [==============================] - 0s 32ms/step - loss: 0.6670 - mean_absolute_error: 0.7208
Epoch 5/200
3/3 [==============================] - 0s 37ms/step - loss: 0.6002 - mean_absolute_error: 0.6875
Epoch 6/200
3/3 [==============================] - 0s 37ms/step - loss: 0.6056 - mean_absolute_error: 0.6746
Epoch 7/200
3/3 [==============================] - 0s 34ms/step - loss: 0.5888 - mean_absolute_error: 0.6896
Epoch 8/200
3/3 [==============================] - 0s 36ms/step - loss: 0.5881 - mean_absolute_error: 0.6716
Epoch 9/200
3/3 [==============================] - 0s 35ms/step - loss: 0.5542 - mean_absolute_error: 0.6479
Epoch 10/200
3/3 [=

<ipython-input-35-2d5bc51efce8>:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Product['Previsão_M1'] = m1
<ipython-input-35-2d5bc51efce8>:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Product['Previsão_M2'] = m2
<ipython-input-35-2d5bc51efce8>:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

Epoch 1/200
3/3 [==============================] - 10s 39ms/step - loss: 0.7919 - mean_absolute_error: 0.7105
Epoch 2/200
3/3 [==============================] - 0s 38ms/step - loss: 0.6519 - mean_absolute_error: 0.6753
Epoch 3/200
3/3 [==============================] - 0s 51ms/step - loss: 0.6139 - mean_absolute_error: 0.6562
Epoch 4/200
3/3 [==============================] - 0s 40ms/step - loss: 0.4907 - mean_absolute_error: 0.6001
Epoch 5/200
3/3 [==============================] - 0s 36ms/step - loss: 0.5070 - mean_absolute_error: 0.6179
Epoch 6/200
3/3 [==============================] - 0s 34ms/step - loss: 0.4986 - mean_absolute_error: 0.6007
Epoch 7/200
3/3 [==============================] - 0s 39ms/step - loss: 0.4968 - mean_absolute_error: 0.5958
Epoch 8/200
3/3 [==============================] - 0s 38ms/step - loss: 0.5003 - mean_absolute_error: 0.5895
Epoch 9/200
3/3 [==============================] - 0s 37ms/step - loss: 0.4609 - mean_absolute_error: 0.5995
Epoch 10/200
3/3 [

<ipython-input-35-2d5bc51efce8>:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Product['Previsão_M1'] = m1
<ipython-input-35-2d5bc51efce8>:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Product['Previsão_M2'] = m2
<ipython-input-35-2d5bc51efce8>:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [ ]:
Boot

,dt_sale,ds_product_line,qt_sale,Previsão_M1,Previsão_M2,Previsão_M3
190,2022-12-01 00:00:00+00:00,Boot,263267.0,269772.375000,269843.062500,269754.937500
192,2023-01-01 00:00:00+00:00,Boot,223118.0,224920.890625,224919.187500,224874.843750
196,2023-02-01 00:00:00+00:00,Boot,215163.0,226787.843750,226788.250000,226787.671875
201,2023-03-01 00:00:00+00:00,Boot,184398.0,235043.687500,235040.265625,235039.796875


In [ ]:
Sandals

,dt_sale,ds_product_line,qt_sale,Previsão_M1,Previsão_M2,Previsão_M3
191,2022-12-01 00:00:00+00:00,Sandals,661730.0,525456.00000,525389.5000,524818.6250
193,2023-01-01 00:00:00+00:00,Sandals,576581.0,520226.78125,522026.0000,521866.4375
197,2023-02-01 00:00:00+00:00,Sandals,573049.0,536253.25000,536114.8125,537886.1875
202,2023-03-01 00:00:00+00:00,Sandals,481353.0,569855.37500,569336.5625,567620.8125


In [ ]:
Shoes

,dt_sale,ds_product_line,qt_sale,Previsão_M1,Previsão_M2,Previsão_M3
189,2022-12-01 00:00:00+00:00,Shoes,1227099.0,1.214072e+06,1.214021e+06,1.214099e+06
194,2023-01-01 00:00:00+00:00,Shoes,1052338.0,9.752893e+05,9.756326e+05,9.771203e+05
198,2023-02-01 00:00:00+00:00,Shoes,944811.0,7.652919e+05,7.657587e+05,7.660679e+05
200,2023-03-01 00:00:00+00:00,Shoes,812154.0,7.810138e+05,7.817388e+05,7.832118e+05


In [ ]:
Sneakers

,dt_sale,ds_product_line,qt_sale,Previsão_M1,Previsão_M2,Previsão_M3
188,2022-12-01 00:00:00+00:00,Sneakers,564820.0,562232.50000,562193.18750,560747.12500
195,2023-01-01 00:00:00+00:00,Sneakers,521048.0,479190.50000,479203.09375,478965.00000
199,2023-02-01 00:00:00+00:00,Sneakers,511301.0,456582.71875,456539.28125,456611.59375
203,2023-03-01 00:00:00+00:00,Sneakers,441090.0,466623.31250,466140.09375,466529.93750


### Utilizando o Prophet

In [ ]:
#Prophet
#Separando em Treino e Teste
data_limite = pd.to_datetime("2023-02-01").date()

#set de treino
train_ts = df_boot[:data_limite]
train_ts = train_ts.drop('ds_product_line', axis = 1)
#set de teste
test_ts = df_boot[data_limite:]
test_ts = test_ts.drop(columns='ds_product_line')

print("Tamanho do conjunto de treino: ", train_ts.shape)
print("Tamanho do conjunto de teste:  ", test_ts.shape)

In [ ]:
# ajuste do dataset de treino
train_ts = train_ts.reset_index()
train_ts.columns = ['ds', 'y']
train_ts.tail()

In [ ]:
# Ajuste do dataset de teste
test_ts = test_ts.reset_index()
test_ts.columns = ['ds', 'y']
print(test_ts.head())
print(test_ts.tail())

In [ ]:
m = Prophet()
m.fit(train_ts)

In [ ]:
# construção do dataset de teste para alimentar o modelo
#future = m.make_future_dataframe(periods=len(test_ts), freq = 'MS') # freq MS = períodos mensais
future = m.make_future_dataframe(periods=1, freq = 'MS')
# realiza previsão
forecast = m.predict(future)

In [ ]:
# plot dos resultados
m.plot(forecast);

In [ ]:
# plot das componentes da série
m.plot_components(forecast);

In [ ]:
# função para construir dataset com valores históricos e previsão realizada
def make_comparison_dataframe(historical, forecast):
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [ ]:
# usa função para construir dataset
prophet_forecast_1 = make_comparison_dataframe(pd.concat((train_ts, test_ts), axis = 0), forecast)

prophet_forecast_1.tail()

In [ ]:
# função reutilizável para cálculo de erros
def calculate_forecast_errors(df, prediction_size):

    df = df.copy()

    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']

    predicted_part = df[-prediction_size:]

    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))

    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

In [ ]:
# cálculo de erros
for err_name, err_value in calculate_forecast_errors(prophet_forecast_1, len(test_ts)).items():
    print(err_name, err_value)
    #Mean absolute percentage error
    #Mean absolute error

In [ ]:
# plot dos resultados
fig = plt.figure()
plt.title("Resultados 1")
plt.plot(train_ts['ds'],train_ts['y'], color = 'black', label='Treino')
plt.plot(test_ts['ds'],test_ts['y'],color='orange', label='Real')
plt.plot(prophet_forecast_1['yhat'], color='red', label='Prophet')
fig.autofmt_xdate()
leg = plt.legend()

In [ ]:
#yhat = previsto e y = valor real da base
prophet_forecast_1.tail()